In [ ]:
  if (returned_i == 2) {
   
   Gi_001B = 0;
   Gd_001A = Id_0038;
   Gi_0051 = HistoryTotal() - 1;
   Gi_001C = Gi_0051;
   if (Gi_0051 >= 0) {
   do { 
   if (!OrderSelect(Gi_001C, 0, 1)) {
   Gd_0018 = -3;
   break;//
   }
   if (OrderSymbol() == _Symbol && OrderMagicNumber() == Ii_0084) { 
   Gl_0051 = OrderCloseTime();
   Gl_0052 = Gi_001B;
   if (Gl_0052 < Gl_0051) { 
   Gi_001B = (int)OrderCloseTime();
   if ((OrderProfit() < 0)) { 
   Gd_001A = NormalizeDouble((OrderLots() * Id_0028), (int)Id_0010);
   } 
   else { 
   Gd_001A = Id_0038;
   }}} 
   Gi_001C = Gi_001C - 1;
   } while (Gi_001C >= 0); 
   }}

## 2. When New Candle is open then open the trade !!

### 2. a. Implementation of IsNewCandle()

## 3. CountBotOrders() returns the total number of orders opened by a bot, based on the  magic number 

In [ ]:
int CountBotOrders() 
{
    int orderCount = 0;
    for (int orderPosition = OrdersTotal() - 1; orderPosition >= 0; orderPosition--) 
    {
        OrderSelect(orderPosition, SELECT_BY_POS, MODE_TRADES);
        if (OrderSymbol() != Symbol() || OrderMagicNumber() != MagicNumber) continue;
        if (OrderSymbol() == Symbol() && OrderMagicNumber() == MagicNumber)
        if (OrderType() == OP_SELL || OrderType() == OP_BUY) orderCount++;
    }
    return orderCount;
}



In [ ]:
bool ModifyTP(int orderType, double TakeProfit)
{
   double totalOpenPrice = 0.0;
   int numOrders = 0;
   
   for (int pos = OrdersTotal() - 1; pos >= 0; pos--)
   {
      if (OrderSelect(pos, SELECT_BY_POS, MODE_TRADES))
      {
         if (OrderMagicNumber() == magicNumber && OrderType() == orderType && OrderSymbol() == Symbol())
         {
            totalOpenPrice += OrderOpenPrice();
            numOrders++;
         }
      }
   }
   
   if (numOrders > 0)
   {
      double averageOpenPrice = totalOpenPrice / numOrders;
       
      if (orderType == 0)
          { double newTakeProfit =  averageOpenPrice - TakeProfit * Point ; }
      
      if (orderType == 1)
         { double newTakeProfit =  averageOpenPrice + TakeProfit * Point ; }
   }
    
    
    for (int i = OrdersTotal() - 1; i >= 0; i--)
       {
          if (OrderSelect(i, SELECT_BY_POS, MODE_TRADES))
          {
             if (OrderMagicNumber() == magicNumber && OrderType() == orderType && OrderSymbol() == Symbol())
             {abs
                bool result = OrderModify(OrderTicket(), OrderOpenPrice(), newTakeProfit, OrderStopLoss(), OrderExpiration(), 0);
                if (result == false)
                {
                   Print("Error modifying take profit for order ", OrderTicket(), ": ", GetLastError());
                }
             }
          }
       }
}


In [ ]:
class CTrade {
private:
    double lotSize;
    double maxSlippage;
    double takeProfit;
    double stopLoss;
    int magicNumber;
    string orderSendComment;

public:
    CTrade(double lotSize, double maxSlippage, double takeProfit, double stopLoss, int magicNumber, string orderSendComment) {
        this->lotSize = lotSize;
        this->maxSlippage = maxSlippage;
        this->takeProfit = takeProfit;
        this->stopLoss = stopLoss;
        this->magicNumber = magicNumber;
        this->orderSendComment = orderSendComment;
    }

    int BuyNow() 
    {
        double takeProfit = Ask + this->takeProfit * Point; // Set the take profit gTakeProfit points above the Ask price
        double stopLoss = Bid - this->stopLoss * Point; // Set the stop loss gStopLoss below Bid price

        // Open a buy order with the specified lot size, entry price, and take profit
        int ticket = OrderSend(Symbol(), OP_BUY, this->lotSize, Ask, this->maxSlippage, stopLoss, takeProfit, this->orderSendComment, this->magicNumber, 0, Blue);

        if (ticket > 0) {
            Print("Buy order opened successfully with ticket #", ticket);
            return ticket;
        } else {
            Print("Failed to open buy order with error code #", GetLastError());
            return 0;
        }
    }

    int SellNow() 
    {
        double takeProfit = Bid - this->takeProfit * Point; // Set the take profit TakeProfit points below the Bid price
        double stopLoss = Ask + this->stopLoss * Point; // Set the stop loss gStopLoss above Bid price

        // Open a sell order with the specified lot size, entry price, take profit, and stop loss
        int ticket = OrderSend(Symbol(), OP_SELL, this->lotSize, Bid, this->maxSlippage, stopLoss, takeProfit, this->orderSendComment, this->magicNumber, 0, Red);

        if (ticket > 0) {
            Print("Sell order opened successfully with ticket #", ticket);
            return ticket;
        } else {
            Print("Failed to open sell order with error code #", GetLastError());
            return 0;
        }
    }

    void ModifyBuyTP(double newTakeProfit)
    {
        ModifyTP(OP_BUY, newTakeProfit, this->magicNumber);
    }
    
    void ModifySellTP(double newTakeProfit)
    {
        ModifyTP(OP_SELL, newTakeProfit, this->magicNumber);
    }
    
    void ModifyTP(int orderType, double TakeProfit, double magicNumber)
        {
           double totalOpenPrice = 0.0;
           int numOrders = 0;
           double newTakeProfit ;

           for (int pos = OrdersTotal() - 1; pos >= 0; pos--)
           {
              if (OrderSelect(pos, SELECT_BY_POS, MODE_TRADES))
              {
                 if (OrderMagicNumber() == magicNumber && OrderType() == orderType && OrderSymbol() == Symbol())
                 {
                    totalOpenPrice += OrderOpenPrice();
                    numOrders++;
                 }
              }
           }


           double averageOpenPrice = totalOpenPrice / numOrders;

           if (orderType == OP_BUY)
           {
              newTakeProfit =  averageOpenPrice + TakeProfit * Point;
           }
           else if (orderType == OP_SELL)
           {
              newTakeProfit =  averageOpenPrice - TakeProfit * Point;
           }

           for (int i = OrdersTotal() - 1; i >= 0; i--)
           {
              if (OrderSelect(i, SELECT_BY_POS, MODE_TRADES))
              {
                 if (OrderMagicNumber() == magicNumber && OrderType() == orderType && OrderSymbol() == Symbol())
                 {
                    bool result = OrderModify(OrderTicket(), OrderOpenPrice(), newTakeProfit, OrderStopLoss(), OrderExpiration(), 0);
                    if (result == false)
                    {
                       Print("Error modifying take profit for order ", OrderTicket(), ": ", GetLastError());
                    }
                 }
              }
           }
        }
